In [19]:
!pip install mediapipe opencv-python pandas scikit-learn

In [20]:
import mediapipe as mp #Import Mediapipe
import cv2 #Import OpenCV

In [21]:
mp_drawing = mp.solutions.drawing_utils #To use the Drawing utilities
mp_holistic = mp.solutions.holistic #Media Pipe Models/Solutions

In [77]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        #Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Make Detection
        results = holistic.process(image) 

        #face_landmarks,pose_landmarks,left_hand_landmarks,right_hand_landmarks

        #Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Draw Face Landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                                  mp_drawing.DrawingSpec(color=(10,110,10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1,circle_radius=1)
                                 )
        
        
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [22]:
results.face_landmarks

NameError: name 'results' is not defined

In [23]:
#Capture Landmarks & Export to CSV

import csv
import os
import numpy as np

In [24]:
num_of_coordinates = len(results.face_landmarks.landmark) + len(results.pose_landmarks.landmark)
num_of_coordinates

NameError: name 'results' is not defined

In [25]:
landmarks = ['class']
for val in range(1, num_of_coordinates+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val),'v{}'.format(val)]
    

NameError: name 'num_of_coordinates' is not defined

In [26]:
landmarks[:-4]

[]

In [84]:
 with open('coords.csv', mode='w', newline='') as f: #creating a file to save the coordinates, mode set to write
     csv_writer = csv.writer(f, delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL) 
     csv_writer.writerow(landmarks) 

In [97]:
class_name = "Confused" 

In [98]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        #Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Make Detection
        results = holistic.process(image) 

        #face_landmarks,pose_landmarks,left_hand_landmarks,right_hand_landmarks

        #Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Draw Face Landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                  mp_drawing.DrawingSpec(color=(10,110,10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1,circle_radius=1)
                                 )
        
        
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )


        #Exporting the coordinates
        try:
            #Extract Pose Landmarks
            pose = results.pose_landmarks.landmark #Extracting all the landmarks for the pose
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            #Extract Face Landmarks
            face = results.face_landmarks.landmark #Extracting all the landmarks for the face
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            #Concateneting the rows
            row = pose_row+face_row

            #Append class name
            row.insert(0, class_name)

            #Export to CSV
            with open('coords.csv',mode='a',newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
        except:
            pass
            
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [27]:
#Train Custom Model Using Scikit Learn

#Read in Collected Data and process

import pandas as pd
from sklearn.model_selection import train_test_split

C:\Users\aqdha\AppData\Local\Temp\ipykernel_18860\3058248387.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [28]:
df = pd.read_csv('coords.csv')


In [29]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Happy,0.564355,0.683652,-1.782059,0.999262,0.597632,0.588651,-1.706622,0.998603,0.619850,...,-0.013514,0.0,0.651144,0.575507,0.015043,0.0,0.657962,0.567276,0.016027,0.0
1,Happy,0.563505,0.681357,-1.606536,0.999269,0.598105,0.587062,-1.505441,0.998662,0.620520,...,-0.012681,0.0,0.643762,0.575438,0.014152,0.0,0.650649,0.568308,0.015051,0.0
2,Happy,0.559015,0.681188,-1.628273,0.999229,0.597439,0.586980,-1.532962,0.998677,0.620308,...,-0.012255,0.0,0.641331,0.573960,0.014589,0.0,0.648214,0.566794,0.015520,0.0
3,Happy,0.558325,0.681500,-1.645875,0.999139,0.597388,0.586906,-1.555972,0.998682,0.620294,...,-0.012758,0.0,0.639646,0.572824,0.013475,0.0,0.646567,0.565413,0.014315,0.0
4,Happy,0.556916,0.680037,-1.651976,0.999067,0.596629,0.584231,-1.562899,0.998672,0.619885,...,-0.010975,0.0,0.638034,0.573288,0.015562,0.0,0.645158,0.565599,0.016501,0.0


In [30]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
394,Confused,0.510711,0.636965,-1.030197,0.999367,0.529802,0.567487,-0.952645,0.999202,0.547352,...,-0.011338,0.0,0.564462,0.588441,0.005442,0.0,0.569201,0.584051,0.005692,0.0
395,Confused,0.511834,0.636815,-0.952709,0.999417,0.531988,0.567373,-0.885684,0.999267,0.549271,...,-0.009148,0.0,0.569903,0.587830,0.008505,0.0,0.574490,0.583288,0.008941,0.0
396,Confused,0.509879,0.637789,-0.773511,0.999466,0.531532,0.569055,-0.704336,0.999330,0.548979,...,-0.009542,0.0,0.574309,0.581076,0.009231,0.0,0.578717,0.576579,0.009746,0.0
397,Confused,0.511354,0.635536,-0.816125,0.999508,0.534157,0.568260,-0.745160,0.999386,0.551190,...,-0.006073,0.0,0.583730,0.579788,0.013525,0.0,0.588468,0.574582,0.014313,0.0
398,Confused,0.515054,0.625702,-0.772005,0.999552,0.538446,0.561100,-0.700581,0.999442,0.554437,...,-0.007499,0.0,0.589302,0.569527,0.013878,0.0,0.593593,0.564762,0.014697,0.0


In [31]:
df[df['class']=='Happy']


,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,Happy,0.564355,0.683652,-1.782059,0.999262,0.597632,0.588651,-1.706622,0.998603,0.619850,...,-0.013514,0.0,0.651144,0.575507,0.015043,0.0,0.657962,0.567276,0.016027,0.0
1,Happy,0.563505,0.681357,-1.606536,0.999269,0.598105,0.587062,-1.505441,0.998662,0.620520,...,-0.012681,0.0,0.643762,0.575438,0.014152,0.0,0.650649,0.568308,0.015051,0.0
2,Happy,0.559015,0.681188,-1.628273,0.999229,0.597439,0.586980,-1.532962,0.998677,0.620308,...,-0.012255,0.0,0.641331,0.573960,0.014589,0.0,0.648214,0.566794,0.015520,0.0
3,Happy,0.558325,0.681500,-1.645875,0.999139,0.597388,0.586906,-1.555972,0.998682,0.620294,...,-0.012758,0.0,0.639646,0.572824,0.013475,0.0,0.646567,0.565413,0.014315,0.0
4,Happy,0.556916,0.680037,-1.651976,0.999067,0.596629,0.584231,-1.562899,0.998672,0.619885,...,-0.010975,0.0,0.638034,0.573288,0.015562,0.0,0.645158,0.565599,0.016501,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,Happy,0.494881,0.617090,-1.379769,0.998953,0.529151,0.542358,-1.298953,0.998527,0.553126,...,-0.010748,0.0,0.569488,0.564536,0.007269,0.0,0.575234,0.558903,0.007701,0.0
138,Happy,0.493839,0.616802,-1.360368,0.998958,0.527502,0.542422,-1.279309,0.998564,0.551600,...,-0.010603,0.0,0.567309,0.561896,0.007045,0.0,0.573142,0.556048,0.007450,0.0
139,Happy,0.492326,0.616884,-1.353896,0.999010,0.524978,0.542492,-1.271813,0.998637,0.549093,...,-0.009638,0.0,0.561268,0.558022,0.007956,0.0,0.567126,0.551611,0.008497,0.0
140,Happy,0.490883,0.617362,-1.349394,0.999034,0.522964,0.542759,-1.268072,0.998681,0.547162,...,-0.009509,0.0,0.558736,0.555647,0.008666,0.0,0.564503,0.549724,0.009119,0.0


In [32]:
x = df.drop('class', axis=1) #features
y = df['class'] #target

In [33]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3, random_state=1234)

In [34]:
y_test

40        Happy
44        Happy
260         Sad
188         Sad
320    Confused
         ...   
167         Sad
220         Sad
283         Sad
225         Sad
266         Sad
Name: class, Length: 120, dtype: object

In [35]:
#Training the machine learning classification model
from sklearn.pipeline import make_pipeline  #making the ML Pipeline.
from sklearn.preprocessing import StandardScaler #Standardized the data

from sklearn.linear_model import LogisticRegression, RidgeClassifier 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [36]:
pipelines = {
    'lr' : make_pipeline(StandardScaler(), LogisticRegression()),
    'rc': make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf': make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb': make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [37]:
pipelines.keys()

dict_keys(['lr', 'rc', 'rf', 'gb'])

In [112]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model =  pipeline.fit(x_train,y_train) #train the model
    fit_models[algo] = model

In [113]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [114]:
fit_models['rc'].predict(row)

C:\Users\aqdha\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: could not convert string to float: 'Confused'

In [12]:
from sklearn.metrics import accuracy_score # Accuracy Metrics To Evaluate the model
import pickle

In [13]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

NameError: name 'fit_models' is not defined

In [14]:
fit_models['rc'].predict(x_test)


NameError: name 'fit_models' is not defined

In [15]:
y_test

NameError: name 'y_test' is not defined

In [16]:
with open('body_language.pkl','wb') as f:
    pickle.dump(fit_models['rf'], f)

NameError: name 'fit_models' is not defined

In [17]:
#Making Detection with the Model
with open('body_language.pkl', 'rb') as f: #rb - read binary 
    model = pickle.load(f) 
    

EOFError: Ran out of input

In [18]:
model

NameError: name 'model' is not defined

In [121]:
from collections import Counter

In [1]:
cap = cv2.VideoCapture(0)
emotions = [] 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        #Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Make Detection
        results = holistic.process(image) 

        #face_landmarks,pose_landmarks,left_hand_landmarks,right_hand_landmarks

        #Recolor image back to BGR for rendering
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Draw Face Landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                                  mp_drawing.DrawingSpec(color=(110,110,10), thickness=1, circle_radius=1),
                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1,circle_radius=1)
                                 )
        
        
        # Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )


        #Exporting the coordinates
        try:
            #Extract Pose Landmarks
            pose = results.pose_landmarks.landmark #Extracting all the landmarks for the pose
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            #Extract Face Landmarks
            face = results.face_landmarks.landmark #Extracting all the landmarks for the face
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

            #Concateneting the rows
            row = pose_row+face_row

           
            # #Append class name
            # row.insert(0, class_name)

            # #Export to CSV
            # with open('coords.csv',mode='a',newline='') as f:
            #     csv_writer = csv.writer(f, delimiter=',', quotechar='"',quoting=csv.QUOTE_MINIMAL)
            #     csv_writer.writerow(row)

            X = pd.DataFrame([row])
            body_language_class  = model.predict(X)[0] #Making detections with the class
            body_language_prob = model.predict_proba(X)[0] #Predicting the probability of the partciular class 
            print(body_language_class, body_language_prob)

            #Declaring the list to store the emotions 
            emotions.append(body_language_class)
           
            #Grab ear coords
            coords = tuple(np.multiply(
                np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, #Grabbing the result value and accesing the pose landmarks coordinates for the left ear 
                          results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y,
                        )), [640,480]).astype(int))    


            
            #Rendering the prediction 
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20,coords[1]-30),
                          (245,117,16),-1)
            cv2.putText(image,body_language_class,coords,
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)


            # Display the probability and the class that is detected
            # Get Status Box
            cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)
            
            # Display class
            cv2.putText(image, 'CLASS',
                        (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class,  # Corrected typo here
                        (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
            # Display probability
            cv2.putText(image, 'PROB',
                        (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)], 2)),  # Corrected typo here
            (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            

        except:
            pass
            
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


# Counting the number of emotions 
emotion_count = Counter(emotions)
most_repeated_emotion = emotion_count.most_common(1)[0][0]

# Provide Relevant Feedback Based on the emotion

print(emotions)

if most_repeated_emotion == 'Happy':
    print("Brilliant! You were maintaining a Positive Body Language Throughout the Interview")
elif most_repeated_emotion == 'Sad':
    print("Oops! Facing an interview with an unpleasant would never land you a job. Be pleasant enough")
elif most_repeated_emotion == 'BadPosture':
    print("You've been maintaining a bad posture throughout the whole interview!")
elif most_repeated_emotion == 'Nervous':
    print("You've Been Nervous The Whole Interview")
elif most_repeated_emotion == 'Confused':
    print("Don't be confused throughout an interview")
else:
    print(f"You've shown {most_common_emotion} the most. How do you feel about that?")
            




NameError: name 'cv2' is not defined